In [1]:
import pandas as pd
import pyodbc
import pgeocode
import pandas as pd
import requests
import xmltodict
import json
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from dataclasses import dataclass, asdict

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
username = 'vigrose'
password_edw = 'Gryffindor~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)
z = "DSN=eprdods; UID={}; PWD={}".format(username, password_edw)
ODS = pyodbc.connect(z)

In [2]:
dea_raw = pd.read_csv("../../Data/DEA/DEA_Raw.csv", encoding='latin1')

In [4]:
me_query= \
    """ 
    SELECT DISTINCT 
    P.DS_PRCTR_ID,
    P.OTHER_ID AS ME
    FROM 
    ODS.PRACTITIONER_ADDL_IDS_LAYOUT P
    WHERE 
    P.ID_QLFR_TYP_CDE = 38
    """
dea_query= \
    """ 
    SELECT DISTINCT 
    P.DS_PRCTR_ID,
    P.OTHER_ID AS DEA
    FROM 
    ODS.PRACTITIONER_ADDL_IDS_LAYOUT P
    WHERE 
    P.ID_QLFR_TYP_CDE = 12
    AND
    P.ID_STUS_DESC='ACTIVE'
    """

In [5]:
me_id = pd.read_sql(con=ODS, sql=me_query)
me_id.head()
dea_id = pd.read_sql(con=ODS, sql=dea_query)
dea_id.head()

,DS_PRCTR_ID,DEA
0,S206528982,AA0001115
1,S206307773,AA0009755
2,S208938318,AA0011356
3,S206840754,AA0012221
4,S208591822,AA0058330


In [6]:
me_dea = pd.merge(me_id, dea_id, on ='DS_PRCTR_ID')

In [7]:
DEA = pd.merge(me_dea, dea_raw, left_on='DEA', right_on='DEA_NBR')

In [106]:
AMAIA_100 = pd.read_csv('../../Data/AMAIA/ALLOWED_FAR.csv')
AMAIA_200 = pd.read_csv('../../Data/AMAIA/ALLOWED_FARTHER.csv')

In [8]:
DHC = pd.read_csv('../../Data/DHC/DHC_2021-01.csv')

In [9]:
dea = DEA[['ME', 'DEA', 'ADDRESS1', 'ADDRESS2', 'CITY', 'STATE', 'ZIP']]

In [10]:
def fix_me(me_list):
    '''Add leading zeroes to ME'''
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
AMAIA_100['ME'] = fix_me(AMAIA_100.ME)
AMAIA_200['ME'] = fix_me(AMAIA_200.ME)
dea['ME'] = fix_me(dea.ME)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
dea = dea.drop_duplicates(['ME','ZIP'])

In [17]:
AMAIA_100['ME_NUMBER']=[x[:-1] for x in AMAIA_100.ME]
AMAIA_200['ME_NUMBER']=[x[:-1] for x in AMAIA_200.ME]
dea['ME_NUMBER']=[x[1:] for x in dea.ME]
# bah_1 = pd.merge(AMAIA_100, dea, left_on=['ME', 'STATE_CD'], right_on=['ME', 'STATE'])
# bah_2 = pd.merge(AMAIA_200, dea, left_on=['ME', 'STATE_CD'], right_on=['ME', 'STATE'])

In [18]:
agobah_1 = pd.merge(AMAIA_100, dea, on='ME_NUMBER')
agobah_2 = pd.merge(AMAIA_200, dea, on='ME_NUMBER')

In [28]:
agobah_2[['ZIP','ZIP_OTHER']]

,ZIP,ZIP_OTHER
0,76502,85750
1,29464,29466
2,31406,29466
3,27804,27804
4,66207,66214
...,...,...
355,62702,54601
356,95630,73003
357,58502,81521
358,11219,60004


In [164]:
agobah_1

,PARTY_ID,POST_CD_ID_OTHER,ADDR_1_OTHER,ADDR_2_OTHER,ADDR_3_OTHER,CITY_OTHER,STATE_CD,SRC_POST_KEY,ZIP_OTHER,FROM_DT_OTHER,...,ME_x,ME_NUMBER,ME_y,DEA,ADDRESS1,ADDRESS2,CITY,STATE,ZIP,ZIP_DISTANCE
0,2201599,20931687,6331 E Placita Atiza,NaN,NaN,Tucson,AZ,14395735.0,85750,2019-09-19 21:09:45.677334,...,03508991010,0350899101,00350899101,BJ7844435,5716 ABERDEEN CT,NaN,TEMPLE,TX,76502,794.828424
1,2495429,31069277,3082 Yachtsman Dr,NaN,NaN,Mt Pleasant,SC,24699474.0,29466,2020-04-05 11:28:55.000000,...,04706871450,0470687145,00470687145,FS1320465,1375 HOSPITAL DRIVE,NaN,MOUNT PLEASANT,SC,29464,1.661036
2,2495429,31069277,3082 Yachtsman Dr,NaN,NaN,Mt Pleasant,SC,24699474.0,29466,2020-04-05 11:28:55.000000,...,04706871450,0470687145,00470687145,FS7140178,9100 WHITE BLUFF ROAD,SUITE 201,SAVANNAH,GA,31406,96.795380
3,1937642,14577673,328 Iron Horse Rd,NaN,NaN,Rocky Mount,NC,936993.0,27804,2016-10-17 14:31:59.582792,...,01606720650,0160672065,00160672065,AL8655485,328 IRON HORSE ROAD,NaN,ROCKY MOUNT,NC,27804,0.000000
4,2961723,20156903,419 Shenandoah Dr,NaN,NaN,Gulf Breeze,FL,13572172.0,32561,2016-10-19 09:30:16.331862,...,48401930119,4840193011,04840193011,FF1461653,5151 N. 9TH AVENUE,SACRED HEART HOSPITAL - NICU,PENSACOLA,FL,32504,9.102443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,4560277,31901546,5245 GRANDWOOD PL E,NaN,NaN,LA CROSSE,WI,25550234.0,54601,2020-12-22 00:08:29.000000,...,49598060036,4959806003,04959806003,FG4425460,350 W CARPENTER ST,NaN,SPRINGFIELD,IL,62702,285.780318
394,1868100,31742637,2011 W Danforth Rd,# 514,NaN,Edmond,OK,25331743.0,73003,2020-11-04 00:48:56.000000,...,00515920960,0051592096,00051592096,BS3882962,9580 OAK AVENUE PARKWAY,#7-164,FOLSOM,CA,95630,1314.803677
395,3476343,31813553,551 Lois Dr,NaN,NaN,Fruita,CO,25446454.0,81521,2020-11-06 00:42:32.000000,...,66102050101,6610205010,06610205010,FM1336189,810E ROSSER AVE,NaN,BISMARCK,ND,58502,669.902993
396,4623497,31851515,116 E Brittany Dr,NaN,NaN,Arlington Hts,IL,25484493.0,60004,2020-11-21 04:10:26.000000,...,87501050545,8750105054,08750105054,FA8447650,4802 10TH AVE,NaN,BROOKLYN,NY,11219,731.334013


In [188]:
S_1.columns

Index(['PARTY_ID', 'POST_CD_ID_OTHER', 'ADDR_1_OTHER', 'ADDR_2_OTHER',
       'ADDR_3_OTHER', 'CITY_OTHER', 'STATE_CD', 'SRC_POST_KEY', 'ZIP_OTHER',
       'FROM_DT_OTHER', 'POST_KEY_OTHER', 'PURPOSE_TYPE_DESC',
       'PURPOSE_TYPE_ID', 'PURPOSE_USG_DESC', 'ME', 'ME_NUMBER', 'OTHER_ID',
       'ADDR_LINE_1_TXT', 'ADDR_LINE_2_TXT', 'ADDR_CITY_NAM', 'ADDR_ST_CDE',
       'ADDR_ZIP_CDE', 'ADDR_FRST_TLPHN_NBR', 'ADDR_SCND_TLPHN_NBR'],
      dtype='object')

In [185]:
count = 0
MES = []
for row in S_2.itertuples():
    if int(row.ADDR_ZIP_CDE) == row.ZIP_OTHER:
        if row.ME_NUMBER not in MES:
            count+= 1
            MES.append(row.ME_NUMBER)
print(count)
count/len(S_2.drop_duplicates('ME_NUMBER'))

16


0.12598425196850394

In [190]:
count = 0
MES = []
for row in S_2.itertuples():
    if row.ADDR_ST_CDE == row.STATE_CD:
        if row.ME_NUMBER not in MES:
            count+= 1
            MES.append(row.ME_NUMBER)
print(count)
count/len(S_2.drop_duplicates('ME_NUMBER'))

54


0.4251968503937008

In [24]:
169/len(agobah_2.drop_duplicates('ME_NUMBER'))

0.6353383458646616

In [191]:
def get_zip_distance(zip1, zip2, dist):
    '''Get distances between zipcodes'''
    zip1 = clean_zipcode(zip1)
    zip2 = clean_zipcode(zip2)
    distance = dist.query_postal_code(zip1, zip2)*0.621371
    return distance

def clean_zipcode(zipcode):
    '''Format zipcode'''
    zipcode = str(zipcode)
    zipcode = zipcode.replace(" ", "")
    zipcode = zipcode.replace(".0", "")
    if len(zipcode) == 3:
        zipcode = "00"+zipcode
    if len(zipcode) == 4:
        zipcode = "0"+zipcode
    else:
        zipcode = zipcode[0:5]
    return zipcode

def get_zip_distances(df):
    '''Get all zip distances'''
    dists = []
    dist = pgeocode.GeoDistance('US')
    for row in df.itertuples():
        dists.append(get_zip_distance(row.ADDR_ZIP_CDE, row.ZIP_OTHER, dist))
    df['ZIP_DISTANCE'] = dists
    return df

In [195]:
with_distance = get_zip_distances(S_2)

In [196]:
with_distance[with_distance.ZIP_DISTANCE <100].drop_duplicates('ME_NUMBER')

,PARTY_ID,POST_CD_ID_OTHER,ADDR_1_OTHER,ADDR_2_OTHER,ADDR_3_OTHER,CITY_OTHER,STATE_CD,SRC_POST_KEY,ZIP_OTHER,FROM_DT_OTHER,...,ME_NUMBER,OTHER_ID,ADDR_LINE_1_TXT,ADDR_LINE_2_TXT,ADDR_CITY_NAM,ADDR_ST_CDE,ADDR_ZIP_CDE,ADDR_FRST_TLPHN_NBR,ADDR_SCND_TLPHN_NBR,ZIP_DISTANCE
2,2641897,13097451,15720 Cumberland St,NaN,NaN,Riverview,MI,2990574.0,48193,2016-10-17 15:26:39.560985,...,4951764008,4951764008,15720 CUMBERLAND ST,None,RIVERVIEW,MI,48193,(734) 258-8007,None,0.000000
5,2751868,12159771,505 W Oak St Ste 202,NaN,NaN,Kissimmee,FL,3313214.0,34741,2016-10-17 15:35:27.189781,...,7480266081,7480266081,800 N CENTRAL AVE,None,KISSIMMEE,FL,34741,None,None,0.000000
6,2679726,15855888,4 Barksdale Ct,NaN,NaN,Hilton Head,SC,3399141.0,29926,2016-10-17 15:37:45.436062,...,8470573007,8470573007,4 BARKSDALE CT,None,HILTON HEAD ISLAND,SC,29926,(843) 689-9225,None,0.000000
7,2752133,12667592,1511 Picardae Ct,NaN,NaN,Powell,OH,5074412.0,43065,2016-10-17 16:20:09.537884,...,5830264061,5830264061,1101 CLUBVIEW BLVD S,None,COLUMBUS,OH,43235,None,None,8.046875
8,2649667,11510091,1018 Sleepy Hollow Rd,NaN,NaN,Clarks Summit,PA,5792806.0,18411,2016-10-17 16:58:28.019656,...,4950567002,4950567002,1018 SLEEPY HOLLOW RD,None,CLARKS SUMMIT,PA,18411,None,None,0.000000
9,2824845,12891841,2344 Smokehill Dr,NaN,NaN,Columbus,OH,9501037.0,43228,2016-10-17 21:46:00.831024,...,7040992002,7040992002,272 HOSPITAL RD,None,CHILLICOTHE,OH,45601,(740) 779-8572,None,43.912282
10,2706527,13792665,95 Fox Hedge Rd,NaN,NaN,Saddle River,NJ,7823107.0,7458,2016-10-17 20:24:44.318316,...,3850165005,3850165005,308 WILLOW AVE,None,HOBOKEN,NJ,07030,(201) 945-2481,None,20.992244
11,2831802,13151143,2674 Laquinta Dr,NaN,NaN,Bay City,MI,7522061.0,48706,2016-10-17 20:08:41.011396,...,5830366022,5830366022,4672 HILL ST,None,CASS CITY,MI,48726,None,None,39.068032
12,2661721,17358091,2820 Canal St,Second Floor,NaN,New Orleans,LA,7951497.0,70119,2016-10-17 20:30:54.657306,...,6491474208,6491474208,2820 CANAL ST,FL 2ND,NEW ORLEANS,LA,70119,(504) 821-8158,None,0.000000
13,2745998,12044817,1407 Tomlin Dr,NaN,NaN,Burr Ridge,IL,8066635.0,60527,2016-10-17 20:36:21.205757,...,7481168034,7481168034,640 E SAINT CHARLES RD,STE 212,CAROL STREAM,IL,60188,None,None,15.903180


In [197]:
53/len(S_2.drop_duplicates('ME_NUMBER'))

0.41732283464566927

In [40]:
npi_query= \
    """ 
    SELECT DISTINCT 
    I.ME,
    K.KEY_VAL AS NPI_NUMBER
    FROM 
    AMAEDW.PARTY_KEY K, AMAEDW.PARTY_ID_TO_ME_VW I
    WHERE  
    K.KEY_TYPE_ID = 38
    AND
    K.ACTIVE_IND = 'Y'
    AND
    I.PARTY_ID = K.PARTY_ID
    """
npi = pd.read_sql(con=AMAEDW, sql=npi_query)

In [42]:
AMAIA_100 = pd.merge(npi, AMAIA_100, on='ME')
AMAIA_200 = pd.merge(npi, AMAIA_200, on='ME')

In [57]:
DHC['Zip Code']

0         ="00929"
1            33428
2            70403
3            15108
4            84094
            ...   
971753       99202
971754       20190
971755       54956
971756       53792
971757       98004
Name: Zip Code, Length: 971758, dtype: object

In [58]:
AMAIA_100.ZIP_OTHER

0      63146
1      61525
2      27106
3      37659
4       2445
       ...  
394    85262
395    87107
396    32563
397    23059
398     6825
Name: ZIP_OTHER, Length: 399, dtype: int64

In [50]:
DHC['NPI_NUMBER'] = [str(x) for x in DHC.NPI]
pd.merge(DHC, AMAIA_100, on='NPI_NUMBER')

,NPI,ME_x,Additional Office Locations,Phone Number,Zip Code,State,City,Address,First Name,Physician Name,...,CITY_OTHER,STATE_CD,SRC_POST_KEY,ZIP_OTHER,FROM_DT_OTHER,POST_KEY_OTHER,PURPOSE_TYPE_DESC,PURPOSE_TYPE_ID,PURPOSE_USG_DESC,ME_NUMBER
0,1538441175,3.011103e+08,"Glen Mills, PA-Media, PA",888.369.2427,19111,PA,Philadelphia,333 Cottman Ave,Meghan,Meghan Boros,...,Tucson,AZ,11135169.0,85741,2016-10-19 06:15:41.714755,11135169.0,Permanent (Parents),471,No Usage,0030111028
1,1659643070,7.810702e+10,"Daytona Beach, FL",405.949.3349,73112,OK,Oklahoma City,3300 Northwest Expy,Mircea,Mircea Mihu,...,Oklahoma City,OK,22659742.0,73104,2020-04-05 11:28:55.000000,22659742.0,License - Florida,420,No Usage,7810702004
2,1659327195,1.601810e+09,NaN,708.783.2696,60402,IL,Berwyn,3249 Oak Park Ave,Patricia,Patricia Gavin,...,Burr Ridge,IL,878956.0,60527,2016-10-17 14:29:48.114323,878956.0,License - Ohio,447,No Usage,0160181013
3,1366493421,1.720792e+09,NaN,877.626.2707,75093,TX,Plano,4001 W 15th St,Ronald,Ronald Scott,...,Naples,FL,20199335.0,34103,2016-10-24 08:35:08.616032,20199335.0,License - Texas,456,No Usage,0172079249
4,1558319301,1.642912e+09,NaN,229.502.9710,31768,GA,Moultrie,760 26th Ave Se,Barbara,Barbara Ebert,...,Fort Harrison,MT,22278012.0,59636,2020-04-05 11:28:55.000000,22278012.0,License - Idaho,425,No Usage,0164291172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,1356349112,4.113832e+09,NaN,503.813.2000,97239,OR,Portland,3181 Sw Sam Jackson Park Rd,James,James Gajewski,...,Portland,OR,20976753.0,97229,2016-12-13 23:26:02.834103,20976753.0,Home,405,No Usage,0411383177
316,1407134224,4.901091e+09,"Purchase, NY",718.920.2961,10467,NY,The Bronx,111 E 210th St,Mili,Mili Shum,...,Salt Lake Cty,UT,24455251.0,84108,2020-03-24 00:28:58.000000,24455251.0,License - Connecticut,417,No Usage,0490109068
317,1609228568,7.044210e+10,"Memphis, TN",662.772.2980,38671,MS,Southaven,7601 Southcrest Pkwy,Muneeba,Muneeba Qayyum,...,Germantown,TN,23712915.0,38138,2019-07-30 21:13:55.905573,23712915.0,Unknown,466,No Usage,7044210001
318,1093376618,3.515190e+09,NaN,NaN,32114,FL,Daytona Beach,201 N Clyde Morris Blvd,Larissa,Larissa Raymond,...,Ormond Beach,FL,23996641.0,32174,2019-10-23 04:29:43.000000,23996641.0,License - Florida,420,No Usage,0351519033


In [51]:
AMAIA_100_DHC = pd.merge(DHC, AMAIA_100, on='NPI_NUMBER')
AMAIA_200_DHC = pd.merge(DHC, AMAIA_200, on='NPI_NUMBER')

In [61]:
AMAIA_200_DHC['ZIP'] = [int(x.replace('=','').replace('"','')) for x in AMAIA_200_DHC['Zip Code']]
AMAIA_100_DHC['ZIP'] = [int(x.replace('=','').replace('"','')) for x in AMAIA_100_DHC['Zip Code']]

In [63]:
count = 0
for row in AMAIA_100_DHC.itertuples():
    if row.ZIP == row.ZIP_OTHER:
        count+= 1
print(count)
count/len(AMAIA_100_DHC)

18


0.05625

In [66]:
len(AMAIA_100)

399

In [74]:
@dataclass
class NumberSearch:
    number: int = ''
    version: str = '2.1'
    enumeration_type: any = 'NPI-1'
def get_results(search):
    url = 'https://npiregistry.cms.hhs.gov/api/?version=2.1'
    parameters = asdict(search)
    response =  requests.get(url, params=parameters)
    try:
        results = response.json()
    except:
        results = False
    return results
def get_npi_info(results):
    new_dict_list =[]
    try:
        result = results['results'][0]
        npi = result['number']
        for address in result['addresses']:
            zipcode = address['postal_code']
            state = address['state']
            new_dict = {
                'NPI': npi,
                'STATE': state,
                'ZIP': zipcode
            }
            new_dict_list.append(new_dict)
    except:
        new_dict_list = False
    return new_dict_list

In [75]:
amaia_100_dict_list = []
for number in AMAIA_100['NPI_NUMBER']:
    results = get_results(NumberSearch(number))
    new_dict = get_npi_info(results)
    if new_dict!=False:
        print(results['results'][0]['basic']['name'])
        amaia_100_dict_list.append(new_dict)

KULAK GARY
ZIA HAMID
JOHNSON TATUM
TUBERVILLE AUDREY
BARBOUR WORTH
MILLAR WILLIAM
ROBINSON MARNIE
SANTORO JOSE
LIM TIMPLE
GIRTON KEITH
LEAHEY ALAN
THOURANI HARICHAND
SCOTT ROBERT
LANDY BRUCE
KANCHERLA DAYAKAR
MONTGOMERY ROBERT
ADAKA GREGORY
GRAVES CHARLES
HAYDU JOSEPH
TAYLOR THOMAS
MODI SUMUL
EASTON TERRY
MOUNZER ASSAAD
HERARD PIERRE
THOMPSON MARK
SOLIS CARLOS
POUAGARE ERSIE
PENN GERALD
KLAPER MICHAEL
ADKINSON DAVID
SASTRY ASHWANI
SMITH MARJORIE
ASHRAF HUSNAIN
ARNOLD TYLER
MAPP JOHN
MCCLARY ASHLEY
RAYMOND LARISSA
SENG HORT
ELLIS CLYDE
KLIOT BENJAMIN
SURAKANTI KEERTHI NANDAN
O'BRYAN MICHAEL
ROBINSON MARTHA
FREYSDOTTIR DRIFA
MURPHY BRENDA
KANNAN NIRMALA
MINIX MICHEAL
TORRES CRISANTO
CHENG DAVID
PANAGAS JAMES
SMITH WILLIAM
MANNING RICHARD
DANIEL RANDY
KRAVCHUK STEPAN
KURBAN WILLIAM
NAGORI MOHAMMEDUMER
DEARY MICHAEL
ELLISON JAMES
BUDHWAR NITIN
MARTIN JAMES
BUTANI ANJALI
BOLDT BRIAN
CHONGTHAM NUNGSHITOMBI
GARG ARVIND
DEBNATH BABUL
MARTINAK BRIDGETTE
AL-JOBURI DINA
ALBETAR RAMI
BARNES CURTIS

In [76]:
amaia_200_dict_list = []
for number in AMAIA_200['NPI_NUMBER']:
    results = get_results(NumberSearch(number))
    new_dict = get_npi_info(results)
    if new_dict!=False:
        print(results['results'][0]['basic']['name'])
        amaia_200_dict_list.append(new_dict)

KULAK GARY
ZIA HAMID
JOHNSON TATUM
TUBERVILLE AUDREY
BARBOUR WORTH
ROBINSON MARNIE
SANTORO JOSE
LIM TIMPLE
GIRTON KEITH
LEAHEY ALAN
THOURANI HARICHAND
SCOTT ROBERT
LANDY BRUCE
KANCHERLA DAYAKAR
MONTGOMERY ROBERT
ADAKA GREGORY
GRAVES CHARLES
HAYDU JOSEPH
TAYLOR THOMAS
MODI SUMUL
EASTON TERRY
HERARD PIERRE
THOMPSON MARK
SOLIS CARLOS
POUAGARE ERSIE
KLAPER MICHAEL
ADKINSON DAVID
SASTRY ASHWANI
SMITH MARJORIE
ASHRAF HUSNAIN
MAPP JOHN
MCCLARY ASHLEY
RAYMOND LARISSA
SENG HORT
ELLIS CLYDE
KLIOT BENJAMIN
SURAKANTI KEERTHI NANDAN
O'BRYAN MICHAEL
ROBINSON MARTHA
MURPHY BRENDA
KANNAN NIRMALA
MINIX MICHEAL
CHENG DAVID
PANAGAS JAMES
SMITH WILLIAM
KRAVCHUK STEPAN
KURBAN WILLIAM
NAGORI MOHAMMEDUMER
DEARY MICHAEL
ELLISON JAMES
BUDHWAR NITIN
BUTANI ANJALI
BOLDT BRIAN
CHONGTHAM NUNGSHITOMBI
GARG ARVIND
DEBNATH BABUL
MARTINAK BRIDGETTE
AL-JOBURI DINA
ALBETAR RAMI
BARNES CURTIS
STOLPE MICHAEL
DEMBLA GEETIKA
HEMLICK JOHN
KANG JANE
RAMOS ABEL
AMIN BABU
CASTRO RICARDO
KHAN MUHAMMAD
SORCE ANGELO
DESMOND CLARE


In [86]:
dict_list_200 = []
for thing in amaia_200_dict_list:
    dict_list_200 = dict_list_200+thing
npi_200 = pd.DataFrame(dict_list_200)
AMAIA_200['NPI']=AMAIA_200.NPI_NUMBER.astype('int64')
NPI_2 = pd.merge(npi_200, AMAIA_200, on='NPI')

In [88]:
dict_list_100 = []
for thing in amaia_100_dict_list:
    dict_list_100 = dict_list_100+thing
npi_100 = pd.DataFrame(dict_list_100)
AMAIA_100['NPI']=AMAIA_100.NPI_NUMBER.astype('int64')
NPI_1 = pd.merge(npi_100, AMAIA_100, on='NPI')

In [93]:
NPI_1.ZIP

0      631465567
1      631465567
2      616370001
3      616370001
4      770242133
         ...    
785    707375023
786    232985025
787    278342818
788    191044399
789    191044319
Name: ZIP, Length: 790, dtype: object

In [94]:
NPI_1.ZIP_OTHER

0      63146
1      63146
2      61525
3      61525
4      27106
       ...  
785    32563
786    23059
787    23059
788     6825
789     6825
Name: ZIP_OTHER, Length: 790, dtype: int64

In [123]:
parties = tuple([str(x)[0:10] for x in AMAIA_100.ME])

In [109]:
AMAIA_100.ME

0       4107590302
1       3508991010
2       1601810138
3       4706871450
4       1606720650
          ...     
506     4706701848
507     4113811752
508    87501050545
509    49637060081
510     4706200725
Name: ME, Length: 511, dtype: int64

In [124]:
iqvia_query = \
        F""" 
        SELECT DISTINCT 
        B.BUSINESS_NAME as "Oooh",
        B.DBA_NAME,
        B.PHONE,
        B.PHYSICAL_ADDR_1,
        B.PHYSICAL_ADDR_2,
        B.PHYSICAL_CITY,
        B.PHYSICAL_STATE,
        B.PHYSICAL_ZIP,
        B.POSTAL_ADDR_1,
        B.POSTAL_ADDR_2,
        B.POSTAL_CITY,
        B.POSTAL_STATE,
        B.POSTAL_ZIP,
        P.ME,
        T.AFFIL_TYPE_DESC,
        A.AFFIL_IND,
        A.AFFIL_RANK
        FROM 
        ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P, ODS.ODS_IMS_AFFILIATION_TYPE T
        WHERE  
        B.IMS_ORG_ID = A.IMS_ORG_ID
        AND
        A.PROFESSIONAL_ID = P.PROFESSIONAL_ID
        AND
        A.AFFIL_TYPE_ID = T.AFFIL_TYPE_ID
        AND
        P.CURRENT_BATCH_FLAG='Y'
        AND
        A.CURRENT_BATCH_FLAG='Y'
        AND
        B.CURRENT_BATCH_FLAG='Y'
        AND
        P.ME in {parties}
        """

In [222]:
iqvia_query_2 = \
        F""" 
        SELECT *
        FROM 
        ODS.ODS_IMS_BUSINESS B
        WHERE  
        B.CURRENT_BATCH_FLAG='Y'
        """

In [244]:
iqvia_query_3 = \
        F""" 
        SELECT *
        FROM 
        ODS.ODS_IMS_PROFESSIONAL P
        WHERE  
        P.CURRENT_BATCH_FLAG='Y'
        """

In [224]:
q = "DSN=eprdods; UID={}; PWD={}".format(username, password_edw)
ODS = pyodbc.connect(q)

In [125]:
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)

In [225]:
iqvia_business = pd.read_sql(con=ODS, sql=iqvia_query_2)

In [245]:
iqvia_professional = pd.read_sql(con=ODS, sql=iqvia_query_3)

In [128]:
AMAIA_100['ME_NUMBER']=[str(x)[0:10] for x in AMAIA_100.ME]
AMAIA_200['ME_NUMBER']=[str(x)[0:10] for x in AMAIA_200.ME]

In [130]:
I_1 = pd.merge(AMAIA_100, iqvia, left_on='ME_NUMBER', right_on='ME')
I_2 = pd.merge(AMAIA_200, iqvia, left_on='ME_NUMBER', right_on='ME')

In [136]:
I_1['ZIP'] = I_1.PHYSICAL_ZIP
I_2['ZIP'] = I_2.PHYSICAL_ZIP

In [176]:
symphony_query = \
    F"""
    SELECT DISTINCT 
    L.OTHER_ID,
    D.ADDR_ST_CDE,
    D.ADDR_ZIP_CDE,
    FROM 
    ODS.PRACTITIONER_DEMOGRAPHIC_LAYOUT D, ODS.PRACTITIONER_ADDL_IDS_LAYOUT L
    WHERE  
    D.DS_PRCTR_ID=L.DS_PRCTR_ID
    AND
    D.ADDR_LINE_1_TXT IS NOT NULL
    AND
    L.ID_QLFR_TYP_CDE=38
    AND
    L.OTHER_ID in {parties}
    """

In [177]:
symphony = pd.read_sql(con=ODS, sql=symphony_query)
symphony.head()

,OTHER_ID,ADDR_LINE_1_TXT,ADDR_LINE_2_TXT,ADDR_CITY_NAM,ADDR_ST_CDE,ADDR_ZIP_CDE,ADDR_FRST_TLPHN_NBR,ADDR_SCND_TLPHN_NBR
0,1320190043,1 LINWOOD AVE W,None,PARAMUS,NJ,07652,(201) 447-8377,None
1,7370698019,100 E IDAHO ST,None,BOISE,ID,83712,(208) 381-2711,None
2,7810702004,1000 N LEE AVE,None,OKLAHOMA CITY,OK,73102,(405) 272-6406,None
3,4220198339,101 E STATE HIGHWAY 260,STE G,PAYSON,AZ,85541,(928) 478-8905,None
4,5831291001,101 MANNING DR,None,CHAPEL HILL,NC,27514,(919) 966-3290,(919) 843-8741


In [180]:
S_1 = pd.merge(AMAIA_100, symphony, left_on='ME_NUMBER',right_on='OTHER_ID')
S_2 = pd.merge(AMAIA_200, symphony, left_on='ME_NUMBER',right_on='OTHER_ID')

In [198]:
iqvia

,Oooh,DBA_NAME,PHONE,PHYSICAL_ADDR_1,PHYSICAL_ADDR_2,PHYSICAL_CITY,PHYSICAL_STATE,PHYSICAL_ZIP,POSTAL_ADDR_1,POSTAL_ADDR_2,POSTAL_CITY,POSTAL_STATE,POSTAL_ZIP,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK
0,HOBOKEN UNIVERSITY MEDICAL CENTER,None,2014181000,308 WILLOW AVE,None,HOBOKEN,NJ,07030-3808,308 WILLOW AVE,None,HOBOKEN,NJ,07030-3808,3850165005,Attending,A,6
1,THE GEORGE WASHINGTON UNIVERSITY HOSPITAL,None,2027154000,900 23RD ST NW,None,WASHINGTON,DC,20037-2342,900 23RD ST NW,None,WASHINGTON,DC,20037-2342,4957007014,Admitting,A,7
2,"EMANUEL MEDICAL CENTER, INC",None,2096674200,825 DELBON AVE,None,TURLOCK,CA,95382-2016,825 DELBON AVE,None,TURLOCK,CA,95382-2016,4955981002,Attending,P,9
3,UNIVERSITY HOSPITAL,None,2103584000,4502 MEDICAL DR,None,SAN ANTONIO,TX,78229-4402,4502 MEDICAL DR,None,SAN ANTONIO,TX,78229-4402,2640193009,IDN Affiliated (Inferred),A,8
4,THE CHILDRENS HOSPITAL OF SAN ANTONIO,None,2107042011,333 N SANTA ROSA ST,None,SAN ANTONIO,TX,78207-3108,333 N SANTA ROSA ST,None,SAN ANTONIO,TX,78207-3108,3190491003,Attending,P,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,TEXAS HEALTH SEAY BEHAVIORAL HEALTH HOSPITAL P...,None,9729818300,6110 W PARKER RD,None,PLANO,TX,75093-7912,6110 W PARKER RD,None,PLANO,TX,75093-7912,7042001022,Admitting,P,10
146,"NORTH SHORE MEDICAL CENTER, INC",NORTH SHORE MEDICAL CENTER SALEM CAMPUS,9787411200,81 HIGHLAND AVE,None,SALEM,MA,01970-2714,81 HIGHLAND AVE,None,SALEM,MA,01970-2714,7591808002,Admitting,P,9
147,COVENANT HEALTHCARE,COVENANT MEDICAL CENTER HARRISON,9895830000,1447 N HARRISON ST,None,SAGINAW,MI,48602-4727,1447 N HARRISON ST,None,SAGINAW,MI,48602-4727,2861299001,Attending,A,8
148,SELECT SPECIALTY HOSPITAL SAGINAW,None,9895834850,1447 N HARRISON ST,None,SAGINAW,MI,48602-4727,1447 N HARRISON ST,None,SAGINAW,MI,48602-4727,2861299001,Admitting (Inferred),A,8


In [207]:
credential = pd.read_excel('../../Data/Credentialling/Org_Addresses.xlsx')

In [208]:
credential =  credential.fillna('None')

In [214]:
credential.columns

Index(['Customer Number', 'Company Name', 'Attention First', 'Attention Last',
       'Street 1ST', 'Street 2ND', 'Street 3RD', 'City', 'State', 'Zip Code',
       'Phone Number', 'Email Address'],
      dtype='object')

In [215]:
credential = credential.fillna('None')
credential['Street 1ST']=[x.upper() for x in credential['Street 1ST']]
credential['City']=[x.upper() for x in credential['City']]
credential['State']=[x.upper() for x in credential['State']]

In [217]:
pd.merge(credential, iqvia, left_on=['Street 1ST', 'City','State',], right_on=['PHYSICAL_ADDR_1', 'PHYSICAL_CITY','PHYSICAL_STATE'])

,Customer Number,Company Name,Attention First,Attention Last,Street 1ST,Street 2ND,Street 3RD,City,State,Zip Code,...,PHYSICAL_ZIP,POSTAL_ADDR_1,POSTAL_ADDR_2,POSTAL_CITY,POSTAL_STATE,POSTAL_ZIP,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK
0,1750569,Kings County Hospital Center,None,None,451 CLARKSON AVE,None,None,BROOKLYN,NY,112032097,...,11203-2054,451 CLARKSON AVE,None,BROOKLYN,NY,11203-2054,5280106023,Admitting (Inferred),A,8
1,2084529,St. Anthony Medical Center,None,None,1201 S MAIN ST,1201 S Main St,None,CROWN POINT,IN,463078483,...,46307-8481,1201 S MAIN ST,None,CROWN POINT,IN,46307-8481,7040992002,Admitting,A,8
2,2100942,Avera Holy Family Health,None,None,826 N 8TH ST,None,None,ESTHERVILLE,IA,513341598,...,51334-1528,826 N 8TH ST,None,ESTHERVILLE,IA,51334-1528,8750108007,Admitting,A,8
3,2084749,The Memorial Hospital,None,None,3073 WHITE MOUNTAIN HWY,None,None,NORTH CONWAY,NH,038607101,...,03860-7101,3073 WHITE MOUNTAIN HWY,None,NORTH CONWAY,NH,03860-7101,2860675003,Treating,A,8
4,2084749,The Memorial Hospital,None,None,3073 WHITE MOUNTAIN HWY,None,None,NORTH CONWAY,NH,038607101,...,03860-7101,3073 WHITE MOUNTAIN HWY,None,NORTH CONWAY,NH,03860-7101,2860675003,Attending,A,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,2108653,Cox Barton County Hospital,None,None,29 NW 1ST LN,None,None,LAMAR,MO,647598105,...,64759-8105,29 NW 1ST LN,None,LAMAR,MO,64759-8105,6540199069,Attending,P,10
74,105762,Nassau County Medical Center,None,None,2201 HEMPSTEAD TPKE,None,None,EAST MEADOW,NY,115541859,...,11554-1859,2201 HEMPSTEAD TPKE,None,EAST MEADOW,NY,11554-1859,4958599004,Attending,P,9
75,2880662,Continuing Care Hospital At Saint Joseph,None,None,1 SAINT JOSEPH DR,None,None,LEXINGTON,KY,405043742,...,40504-3742,1 SAINT JOSEPH DR,None,LEXINGTON,KY,40504-3742,3050110310,Admitting,P,10
76,2880662,Continuing Care Hospital At Saint Joseph,None,None,1 SAINT JOSEPH DR,None,None,LEXINGTON,KY,405043742,...,40504-3742,1 SAINT JOSEPH DR,None,LEXINGTON,KY,40504-3742,3050110310,Admitting (Inferred),P,10


In [220]:
cred = cred.fillna('None')
cred['address_1'] = [x.upper() for x in cred['address_1']]
cred['city'] = [x.upper() for x in cred['city']]
cred['state'] = [x.upper() for x in cred['state']]
matches = pd.merge(cred, iqvia,left_on=['address_1', 'city', 'state'],right_on=['physical_address_1', 'physical_city', 'physical_state'])
print(matches)
print(cred)
print(iqvia)

    Customer Number                              Company Name Attention First  \
0           1750569              Kings County Hospital Center            None   
1           2084529                St. Anthony Medical Center            None   
2           2100942                  Avera Holy Family Health            None   
3           2084749                     The Memorial Hospital            None   
4           2084749                     The Memorial Hospital            None   
..              ...                                       ...             ...   
73          2108653                Cox Barton County Hospital            None   
74           105762              Nassau County Medical Center            None   
75          2880662  Continuing Care Hospital At Saint Joseph            None   
76          2880662  Continuing Care Hospital At Saint Joseph            None   
77          3425247                          Huggins Hospital            None   

   Attention Last          

In [219]:
cred = credential.rename(columns = {'Street 1ST': 'address_1', 'City':'city','State':'state'})
iqvia = iqvia.rename(columns = {'PHYSICAL_ADDR_1':'physical_address_1', 'PHYSICAL_CITY':'physical_city','PHYSICAL_STATE':'physical_state'})

In [221]:
matches

,Customer Number,Company Name,Attention First,Attention Last,address_1,Street 2ND,Street 3RD,city,state,Zip Code,...,PHYSICAL_ZIP,POSTAL_ADDR_1,POSTAL_ADDR_2,POSTAL_CITY,POSTAL_STATE,POSTAL_ZIP,ME,AFFIL_TYPE_DESC,AFFIL_IND,AFFIL_RANK
0,1750569,Kings County Hospital Center,None,None,451 CLARKSON AVE,None,None,BROOKLYN,NY,112032097,...,11203-2054,451 CLARKSON AVE,None,BROOKLYN,NY,11203-2054,5280106023,Admitting (Inferred),A,8
1,2084529,St. Anthony Medical Center,None,None,1201 S MAIN ST,1201 S Main St,None,CROWN POINT,IN,463078483,...,46307-8481,1201 S MAIN ST,None,CROWN POINT,IN,46307-8481,7040992002,Admitting,A,8
2,2100942,Avera Holy Family Health,None,None,826 N 8TH ST,None,None,ESTHERVILLE,IA,513341598,...,51334-1528,826 N 8TH ST,None,ESTHERVILLE,IA,51334-1528,8750108007,Admitting,A,8
3,2084749,The Memorial Hospital,None,None,3073 WHITE MOUNTAIN HWY,None,None,NORTH CONWAY,NH,038607101,...,03860-7101,3073 WHITE MOUNTAIN HWY,None,NORTH CONWAY,NH,03860-7101,2860675003,Treating,A,8
4,2084749,The Memorial Hospital,None,None,3073 WHITE MOUNTAIN HWY,None,None,NORTH CONWAY,NH,038607101,...,03860-7101,3073 WHITE MOUNTAIN HWY,None,NORTH CONWAY,NH,03860-7101,2860675003,Attending,A,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,2108653,Cox Barton County Hospital,None,None,29 NW 1ST LN,None,None,LAMAR,MO,647598105,...,64759-8105,29 NW 1ST LN,None,LAMAR,MO,64759-8105,6540199069,Attending,P,10
74,105762,Nassau County Medical Center,None,None,2201 HEMPSTEAD TPKE,None,None,EAST MEADOW,NY,115541859,...,11554-1859,2201 HEMPSTEAD TPKE,None,EAST MEADOW,NY,11554-1859,4958599004,Attending,P,9
75,2880662,Continuing Care Hospital At Saint Joseph,None,None,1 SAINT JOSEPH DR,None,None,LEXINGTON,KY,405043742,...,40504-3742,1 SAINT JOSEPH DR,None,LEXINGTON,KY,40504-3742,3050110310,Admitting,P,10
76,2880662,Continuing Care Hospital At Saint Joseph,None,None,1 SAINT JOSEPH DR,None,None,LEXINGTON,KY,405043742,...,40504-3742,1 SAINT JOSEPH DR,None,LEXINGTON,KY,40504-3742,3050110310,Admitting (Inferred),P,10


In [229]:
iqvia_business = iqvia_business.fillna('None')

In [236]:
    iqvia_business_active = iqvia_business[iqvia_business.STATUS_INDICATOR!='D']

In [239]:
len(iqvia_business)

1013767

In [241]:
results = []
for col in iqvia_business.columns:
    COMPLETE = len(iqvia_business[iqvia_business[col]!='None'])
    ACTIVE_COMPLETE = len(iqvia_business_active[iqvia_business_active[col]!='None'])
    new_dict = {
        'COLUMN': col,
        'ACTIVE_COMPLETE': ACTIVE_COMPLETE,
        'TOTAL_COMPLETE': COMPLETE,
        'ACTIVE_COMPLETENESS': round((ACTIVE_COMPLETE/695410)*100, 3),
        'TOTAL_COMPLETENESS': round((COMPLETE/1013767)*100, 3)
    }
    results.append(new_dict)
    print(f"{col} : {new_dict['ACTIVE_COMPLETENESS']}")

IMS_ORG_ID : 100.0
BUSINESS_NAME : 100.0
DBA_NAME : 4.329
CARE_OF_LOCATION : 0.273
ADDRESS_ID : 100.0
STF_ID : 100.0
PHYSICAL_ADDR_1 : 100.0
PHYSICAL_ADDR_2 : 36.36
PHYSICAL_CITY : 100.0
PHYSICAL_STATE : 100.0
PHYSICAL_ZIP : 100.0
POSTAL_ADDR_1 : 100.0
POSTAL_ADDR_2 : 36.36
POSTAL_CITY : 100.0
POSTAL_STATE : 100.0
POSTAL_ZIP : 100.0
PHONE : 95.917
FAX : 63.794
WEBSITE : 64.595
LATITUDE : 99.995
LONGITUDE : 99.995
OWNER_STATUS : 97.773
PROFIT_STATUS : 13.293
CMI : 0.63


C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


PRIMARY_COT_ID : 100.0
COT_CLASSIFICATION_ID : 100.0
COT_CLASSIFICATION : 100.0
COT_FACILITY_TYPE_ID : 100.0
COT_FACILITY_TYPE : 100.0
COT_SPECIALTY_ID : 50.663
COT_SPECIALTY : 50.663
RECORD_TYPE : 100.0
BED_CLUSTER_ID : 0.0
TTL_LICENSE_BEDS : 6.467
TTL_CENSUS_BEDS : 6.467
TTL_STAFFED_BEDS : 6.467
TEACHING_HOSP : 0.188
COMMHOSP : 0.574
ALL_DDD : 0.0
ALL_NPI : 0.0
HIN : 0.0
DEA : 12.907
MPN : 0.914
MPN_ORDER : 0.914
MSA : 94.807
FIPS_STATE : 99.936
FIPS_COUNTY : 99.936
NUM_OF_PROVIDERS : 42.395
CORP_PARENT_IMS_ORG_ID : 36.582
CORP_PARENT_NAME : 36.582
OWNER_SUB_IMS_ORG_ID : 6.667
OWNER_SUB_NAME : 6.667
GPO_PHARMA_IMS_ORG_ID : 20.172
GPO_PHARMA_NAME : 20.172
GPO_MEDSURG_IMS_ORG_ID : 19.748
GPO_MEDSURG_NAME : 19.748
PHARMA_PROV_IMS_ORG_ID : 1.137
PHARMA_PROV_NAME : 1.137
FORMULARY : 25.444
ELECTRONIC_MED_REC : 45.941
EPRESCRIBE : 42.787
PAYPERFORM : 21.32
GENFIRST : 28.147
SREP_ACCESS : 43.547
DEACTIVATION_REASON : 0.0
REFERBACK_IMS_ORG_ID : 0.0
STATUS_INDICATOR : 100.0
JOB_ID : 100.0
CUR

In [243]:
pd.DataFrame(results).to_csv('../../Data/MasterfileCore/IQVia_Business_Column_Completion.csv', index=False)

In [247]:
iqvia_professional.STATUS_DESC

0          Active
1          Active
2          Active
3          Active
4          Active
            ...  
1575600    Active
1575601    Active
1575602    Active
1575603    Active
1575604    Active
Name: STATUS_DESC, Length: 1575605, dtype: object

In [250]:
def get_column_totals(dataframe):
    dataframe = dataframe.fillna('None')
    results = []
    total = len(dataframe)
    for col in dataframe.columns:
        COMPLETE = len(dataframe[dataframe[col]!='None'])
        new_dict = {
            'COLUMN': col,
            'TOTAL_COMPLETE': COMPLETE,
            'TOTAL_COMPLETENESS': round((COMPLETE/total), 5)
        }
        results.append(new_dict)
        print(f"{col} : {new_dict['TOTAL_COMPLETENESS']}")
    return(results)

In [251]:
prof_results = get_column_totals(iqvia_professional)

HCE_ID_PRFSNL : 1.0
PROFESSIONAL_ID : 1.0
PARTY_ID : 0.63573
FIRST_NAME : 1.0
MIDDLE_NAME : 0.60411
LAST_NAME : 1.0
GEN_SUFFIX : 0.02629
DESIGNATION : 0.0
GENDER : 1.0
ROLE : 1.0
PRIMARY_SPEC : 1.0
SECONDARY_SPEC : 0.4096
TERTIARY_SPEC : 0.13484
PRIMARY_PROF_CODE : 1.0
PRIMARY_PROF_DESC : 1.0
IMS_ID : 0.70058
UPIN : 0.29312
ME : 0.63574
ALL_DEA : 0.0
NPI : 0.68214
STATUS_DESC : 1.0
JOB_ID : 1.0
CURRENT_BATCH_FLAG : 1.0
BATCH_BUSINESS_DATE : 1.0


In [253]:
pd.DataFrame(prof_results).to_csv('../../Data/MasterfileCore/IQVia_Professional_Column_Completion.csv', index=False)

In [263]:
add = pd.read_csv('../../Data/MasterfileCore/ProgramAddress.txt', delimiter = '|', encoding='latin')
inst_info = pd.read_csv('../../Data/MasterfileCore/InstitutionInformation.txt', delimiter = '|', encoding='latin', error_bad_lines=False)
prog_info = pd.read_csv('../../Data/MasterfileCore/ProgramInformation.txt', delimiter = '|', encoding='latin', error_bad_lines=False)
prog_inst = pd.read_csv('../../Data/MasterfileCore/ProgramInstitution.txt', delimiter = '|', encoding='latin')
prog_pers = pd.read_csv('../../Data/MasterfileCore/ProgramPersonnel.txt', delimiter = '|', encoding='latin', error_bad_lines=False)

b'Skipping line 601: expected 14 fields, saw 18\n'
b'Skipping line 5168: expected 55 fields, saw 59\nSkipping line 5183: expected 55 fields, saw 106\n'
b'Skipping line 8397: expected 35 fields, saw 36\n'
b'Skipping line 31221: expected 35 fields, saw 52\n'
b'Skipping line 51840: expected 35 fields, saw 57\nSkipping line 51876: expected 35 fields, saw 44\n'


In [266]:
meri_dict_list = [{'table': add,'table_name':'ProgramAddress'},
                 {'table': inst_info,'table_name':'InstitutionInformation'},
                 {'table': prog_info,'table_name':'ProgramInformation'},
                 {'table': prog_inst,'table_name':'ProgramInstitution'},
                 {'table': prog_pers,'table_name':'ProgramPersonnel'}]

In [272]:
for thing in meri_dict_list:
    new_results = get_column_totals(thing['table'])
    pd.DataFrame(new_results).to_csv(f"../../Data/MasterfileCore/{thing['table_name']}_Column_Completion.csv", index=False)

pgm_id : 1.0
addr_type : 1.0
addr1 : 0.63157
addr2 : 0.60374
addr3 : 0.53625
city : 0.63151
state : 0.63155
Zip+4 : 0.63148
survey_cycle : 0.99901
last_upd_dt : 0.54637
last_upd_type : 0.54637
last_upd_id : 0.54637
ins_id : 0.99986
ins_control_type : 0.94236
ins_type : 0.99889
ins_affiliation_type : 0.99986
ins_name : 1.0
ins_name_change_dt : 0.14968
ins_old_name : 0.16249
ins_activity_status : 0.99958
ins_prog_survey_coord : 0.15901
add_date : 0.74534
survey_cycle : 0.90755
last_upd_dt : 0.38081
last_upd_type : 0.3829
last_upd_id : 0.3861
pgm_id : 1.0
pgm_spec : 0.99984
pgm_state : 0.99984
pgm_inst_control_type : 1.0
pgm_sequence_number : 1.0
pgm_federal_code : 0.99992
pgm_region_code : 1.0
pgm_activity_code : 0.99992
prm_acgme_accred_ind : 0.99704
pgm_name : 0.99984
pgm_web_address : 0.93134
pgm_old_name : 0.37369
pgm_name_change_date : 0.3563
pgm_contact_dir_ind : 0.98526
pgm_accred_status : 0.99976
pgm_accred_eff_dt : 0.99968
pgm_init_accred_dt : 0.37289
pgm_accred_length : 0.99984

In [271]:
prog_pers = prog_pers[prog_pers.pers_type=='D']

In [335]:
prog_inst.groupby('pri_clinical_loc_ind').count()

,pgm_id,ins_id,affiliation_type,survey_cycle,last_upd_dt,last_upd_type,last_upd_id
pri_clinical_loc_ind,,,,,,,
0.0,20005,20005,20005,19988,1,1,1
1.0,7574,7574,7574,7536,0,0,0


In [334]:
prog_inst.sort_values('pri_clinical_loc_ind')

,pgm_id,ins_id,affiliation_type,pri_clinical_loc_ind,survey_cycle,last_upd_dt,last_upd_type,last_upd_id
0,200121109,10497,A,0.0,2009.0,NaN,NaN,NaN
24983,3211721006,170141,A,0.0,2010.0,NaN,NaN,NaN
24977,3211632037,160447,A,0.0,2012.0,NaN,NaN,NaN
24976,3211632037,160278,A,0.0,2012.0,NaN,NaN,NaN
24975,3211632037,160164,A,0.0,2012.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
39995,9995500249,559513,A,NaN,2017.0,NaN,NaN,NaN
39996,9995500249,559515,A,NaN,2017.0,NaN,NaN,NaN
40003,9995600184,560237,A,NaN,2015.0,NaN,NaN,NaN
40005,9995600184,560441,S,NaN,2015.0,NaN,NaN,NaN


In [291]:
inst_info

,ins_id,ins_control_type,ins_type,ins_affiliation_type,ins_name,ins_name_change_dt,ins_old_name,ins_activity_status,ins_prog_survey_coord,add_date,survey_cycle,last_upd_dt,last_upd_type,last_upd_id
0,010004,23.0,N22,A,USA Strada Patient Care Center,NaN,NaN,0,NaN,10/28/2019,2019.0,NaN,NaN,NaN
1,010005,23.0,N22,A,West Mobile Medical Group,NaN,NaN,0,NaN,8/3/2020,2020.0,NaN,NaN,NaN
2,010006,23.0,H01,A,Brewton Medical Center,NaN,NaN,0,NaN,8/3/2020,2020.0,NaN,NaN,NaN
3,010008,23.0,N22,A,Tennessee River Dermatology,NaN,NaN,0,NaN,10/15/2019,2019.0,NaN,NaN,NaN
4,010010,23.0,N22,A,Felix Morris MD LLC Office,NaN,NaN,0,NaN,10/15/2019,2019.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7177,570003,16.0,H01,A,Ivinson Memorial Hospital,NaN,NaN,0,NaN,5/28/2019,2019.0,NaN,NaN,NaN
7178,570100,23.0,H01,A,Wyoming Medical Center,8/1/1986,NaN,0,N,NaN,1997.0,NaN,NaN,NaN
7179,570106,13.0,H01,A,Cheyenne Regional Medical Center,2/2/2007,United Medical Center,0,N,NaN,2006.0,2/2/2007,A,cmathews
7180,578001,45.0,H01,A,Cheyenne Veterans Affairs Medical Center,NaN,NaN,0,NaN,6/6/1997,1996.0,NaN,NaN,NaN


In [303]:
pd.merge(prog_info, inst_info, left_on='pgm_prim_site', right_on='ins_id')

,pgm_id,pgm_spec,pgm_state,pgm_inst_control_type,pgm_sequence_number,pgm_federal_code,pgm_region_code,pgm_activity_code,prm_acgme_accred_ind,pgm_name,...,ins_name,ins_name_change_dt,ins_old_name,ins_activity_status,ins_prog_survey_coord,add_date,survey_cycle_y,last_upd_dt_y,last_upd_type_y,last_upd_id_y


In [302]:
prog_info['pgm_prim_site'] = prog_info.fillna(0).pgm_prim_site.astype(int).astype(object)

In [304]:
prog_info.pgm_prim_site

0         10498
1         30509
2         38019
3         40284
4         50554
          ...  
12476    550426
12477    559513
12478    560355
12479    560337
12480    560441
Name: pgm_prim_site, Length: 12481, dtype: object

In [305]:
inst_info.ins_id

0       010004
1       010005
2       010006
3       010008
4       010010
         ...  
7177    570003
7178    570100
7179    570106
7180    578001
7181    579501
Name: ins_id, Length: 7182, dtype: object

In [4]:
z = "DSN=PRDDM; UID={}; PWD={}".format(username, password_edw)
AMADM = pyodbc.connect(z)

In [ ]:
today_phys_query= \
    """ 
    SELECT DISTINCT 
    D.FULL_DT,
    H.MED_EDU_NBR AS ME,
    H.PARTY_ID,
    O.ORDER_NBR,
    O.ORDER_PRODUCT_ID, 
    O.ORDER_PHYSICIAN_HIST_KEY,
    O.CUSTOMER_KEY,
    O.CUSTOMER_NBR
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND
    D.MONTH_NBR = 5
    AND
    D.YR = 2020
    """

In [337]:
cust = \
    """
    SELECT *
    FROM 
    AMADM.dim_customer c  
    """

In [3]:
product = \
    """
    SELECT *
    FROM 
    AMADM.dim_product 
    """

In [362]:
phys = \
    """
    SELECT *
    FROM 
    AMADM.dim_physician_hist
    """

In [6]:
eprofile = \
    """
    SELECT *
    FROM 
    AMADM.FACT_EPROFILE_ORDERS
    """

In [339]:
cred_cust = pd.read_sql(con=AMADM, sql=cust)

In [ ]:
cred_phys = pd.read_sql(con=AMADM, sql=phys)

In [5]:
cred_prod = pd.read_sql(con=AMADM, sql=product)

In [7]:
cred_orders = pd.read_sql(con=AMADM, sql=eprofile)

KeyboardInterrupt: 

In [343]:
cred_cust_totals = get_column_totals(cred_cust)

CUSTOMER_KEY : 1.0
CUSTOMER_NBR : 1.0
CUSTOMER_ISELL_LOGIN : 0.12361
CUSTOMER_NAME : 0.99597
CUSTOMER_TYPE : 1.0
CUSTOMER_TYPE_DESC : 1.0
CUSTOMER_CATEGORY : 0.16191
CUSTOMER_CATEGORY_DESC : 0.16191
CURRENT_IND : 1.0
EFFECTIVE_BEGIN_DT : 1.0
EFFECTIVE_END_DT : 1.0
SOURCE_SYSTEM : 1.0
DW_CREATED_TS : 1.0
DW_UPDATED_TS : 1.0
DW_CREATED_BY_ID : 1.0
DW_UPDATED_BY_ID : 1.0
DW_LOAD_RUN_ID : 1.0


In [347]:
cred_prod = pd.read_sql(con=AMADM, sql=product)

In [344]:
pd.DataFrame(cred_cust_totals).to_csv('../../Data/MasterfileCore/Credentialing_Customer_Column_Completion.csv', index=False)

In [361]:
for bing in cred_prod.PRODUCT_CAT_DESC.unique():
    print(bing)

AMA PRESS
Commission Tracking
Receivables
Dues
Federation
None
AFFINITY PROGRAMS
Publication
Insurance
JAMA Products
AMA Insurance
CPT Services
G/L
Overpayment
AM News Products
OTHER
AMPAC DONATIONS
Members Only Website Access
Profiles


In [360]:
cred_prod.groupby('PRODUCT_CAT_DESC').count()

,PRODUCT_ID,PRODUCT_CD,PRODUCT_DESC,PRODUCT_CAT_CD,INSERT_TMSTP,INSERT_USERID,UPDATE_TMSTP,UPDATE_USERID,PRODUCT_DESC_SHORT,PRODUCT_DESC_LONG,PRODUCT_SOURCE_ID,PRODUCT_SOURCE_NM
PRODUCT_CAT_DESC,,,,,,,,,,,,
AFFINITY PROGRAMS,1927,1927,1927,1927,1927,1927,1927,1927,0,0,1927,1927
AM News Products,20,20,20,20,20,20,20,20,0,0,20,20
AMA Insurance,2,2,2,2,2,2,2,2,0,0,2,2
AMA PRESS,17357,17319,17323,17357,17357,17357,17357,17357,0,0,16431,16431
AMPAC DONATIONS,1,1,1,1,1,1,1,1,0,0,1,1
CPT Services,2,2,2,2,2,2,2,2,0,0,2,2
Commission Tracking,129,129,129,129,129,129,129,129,0,0,129,129
Dues,210,210,210,210,210,210,210,210,0,0,210,210
Federation,63,63,63,63,63,63,63,63,0,0,63,63
